In [1]:
import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 0.3.27


In [2]:
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import os, getpass


load_dotenv()

llm = ChatOpenAI(model="gpt-4.1-mini")


In [10]:
llm_response = llm.invoke("Tell me a proverb about AI")

print(llm_response)

content='"Artificial intelligence is only as wise as the data it learns from."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 13, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BzqG560ZhVfqrBFngI2P8lXwaENt0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--0fa1d227-2666-4aa0-87be-f3bb3e78d88c-0' usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [4]:

chain = llm | output_parser
response = chain.invoke("Tell me a proverb about AI")
print(response)

"AI is a mirror reflecting the wisdom and flaws of its creators."


In [5]:
from pydantic import BaseModel, Field, EmailStr

class ProjectDetails(BaseModel):
    name: str = Field(description="Project Name")
    datacenter: str = Field(description="Datacenter Location")
    projectUsageType: str = Field(description="Type of Project Usage")
    id: str = Field(description="Unique Project Identifier")
    email: EmailStr = Field(description="Lead Engineer's Email")

project_notes = """
 The project titled Apollo Insights was recently initiated to support AI-powered analytics for the finance department.
   This initiative is currently hosted in the AUS datacenter and falls under the CLIENT_TRIAL project usage type. 
   The system was registered under the unique identifier PRJ-908234. Project communication is managed primarily 
   through the lead engineer’s email: ai-leads@apolloinsights.com. As part of the pilot rollout, 
   Apollo Insights will process high-volume telemetry data for regulatory and operational optimization.
"""


structured_llm = llm.with_structured_output(ProjectDetails)
project_details = structured_llm.invoke(project_notes)
print(project_details)
print(project_details.email )


name='Apollo Insights' datacenter='AUS' projectUsageType='CLIENT_TRIAL' id='PRJ-908234' email='ai-leads@apolloinsights.com'
ai-leads@apolloinsights.com


## Prompt Template

In [13]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a fact on {topic}")
# prompt.invoke({"topic" : "Engineering"})


chain = prompt | llm | output_parser
chain.invoke({"topic": "credit cards"})


'A fact about credit cards is that the first universal credit card, which could be used at a variety of stores, was introduced by Diners Club in 1950. This innovation helped pave the way for the widespread use of credit cards today.'

## Full Example

In [4]:
from langchain_core.prompts  import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini")
# Define Prompt
prompt = ChatPromptTemplate.from_template("Tell me a fact on {topic}")
# Define Output Parser
output_parser = StrOutputParser()

#Compose Chain (A Runnable Sequence)
chain = prompt | llm | output_parser

# use chain
response = chain.invoke({"topic": "Transistor"})
print(response)

A transistor is a semiconductor device used to amplify or switch electronic signals and electrical power. It was invented in 1947 by John Bardeen, Walter Brattain, and William Shockley at Bell Labs and is considered one of the most important inventions in modern electronics.


## Ingestion: loading docs and Splitting it into Chunks and embedding the chunks

In [23]:
from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document
import os
from dotenv import load_dotenv

load_dotenv()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=100, 
    length_function=len
)

def load_docs(folder_path: str) -> List[Document]:
    """Load documents from a folder."""
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.txt'):
            loader = TextLoader(file_path)
        elif filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents


folder_path = "../docs"
docs = load_docs(folder_path)
print(f"Loaded {len(docs)} documents from {folder_path}")

# Split documents into chunks
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks")

# Create embeddings for the chunks
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Generated embeddings for {len(document_embeddings)} chunks")
print(document_embeddings[0])  # Print the first embedding vector



Loaded 4 documents from ../docs
Split into 13 chunks
Generated embeddings for 13 chunks
[0.01644730754196644, -0.017504924908280373, 0.02378283441066742, -0.036582715809345245, -0.005183003842830658, 0.018698135390877724, -0.02418961003422737, 0.005074529908597469, -0.014210039749741554, -0.017111709341406822, 0.018481187522411346, 0.004305045586079359, -0.0012347345473244786, -0.00019915073062293231, 0.026874331757426262, 0.007321967277675867, 0.011288032867014408, -0.02763364650309086, -0.004576229490339756, 0.002120319753885269, -0.015132065862417221, -0.012508360669016838, -0.0021220145281404257, 0.012149041518568993, 0.020976079627871513, 0.010067705065011978, -0.00018485001055523753, -0.021532006561756134, 0.011993111111223698, 0.011443963274359703, 0.04227758198976517, -0.016420189291238785, -0.008162637241184711, -0.010745665058493614, -0.019267620518803596, -0.01216260064393282, 0.015701550990343094, -0.018779490143060684, 0.015308335423469543, -0.012671071104705334, 0.0074711

In [24]:
from langchain_chroma import Chroma
import os
import shutil

# Clean up any existing chroma database
chroma_db_path = os.path.abspath("./chroma_db")
if os.path.exists(chroma_db_path):
    print(f"Removing existing database at {chroma_db_path}")
    shutil.rmtree(chroma_db_path)

# Create the directory with proper permissions
os.makedirs(chroma_db_path, exist_ok=True)

embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002")
collection_name = "pineapple_docs"

try:
    vector_store = Chroma.from_documents(
        documents=splits,
        embedding=embedding_function,
        collection_name=collection_name,
        persist_directory=chroma_db_path
    )
    print(f"Vector store created with documents in collection '{collection_name}'")
    print(f"Database persisted at: {chroma_db_path}")
except Exception as e:
    print(f"Error creating vector store: {e}")
    # Fallback: create in-memory vector store
    print("Creating in-memory vector store as fallback...")
    vector_store = Chroma.from_documents(
        documents=splits,
        embedding=embedding_function,
        collection_name=collection_name
    )
    print("In-memory vector store created successfully")

Error creating vector store: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database
Creating in-memory vector store as fallback...
In-memory vector store created successfully
In-memory vector store created successfully


In [25]:
#  Perform Similiarity Search - quick testing and validation. Not for usage in Agent

query = "Who is CEO of Pineapple Inc?"
results = vector_store.similarity_search(query, k=2)
print(f"Found {len(results)} relevant documents for query '{query}':")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.metadata['source']}: {result.page_content[:100]}...")  # Print first 100 chars of content
    print(f"   Metadata: {result.metadata}")  # Print metadata for each result
    print()


Found 2 relevant documents for query 'Who is CEO of Pineapple Inc?':
1. ../docs/pineapple_company.docx: Pineapple Technologies is a fictional tech giant based in Redwood Valley, California.
Founded in 198...
   Metadata: {'source': '../docs/pineapple_company.docx'}

2. ../docs/pineapple_piphone_info.txt: Pineapple Technologies is a fictional tech giant based in Redwood Valley, California.
Founded in 198...
   Metadata: {'source': '../docs/pineapple_piphone_info.txt'}



In [29]:
# This is the retriever code and should be used in the agent
query = "Who is CEO of Pineapple Inc?"
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# below is for testing the retriever
retriever_results = retriever.invoke(query)
print(f"Retriever found {len(retriever_results)} documents for query '{query}':")


Retriever found 2 documents for query 'Who is CEO of Pineapple Inc?':


In [38]:
from langchain_core.runnables import RunnablePassthrough

template = """
You are a helpful assistant. You will answer questions based on the provided context.
Context: {context}
Question: {question}
Answer:
"""


prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Fixed chain structure - the input should be just the question string
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | output_parser
)

question = "Who is the CEO of Pineapple Inc?"
response = chain.invoke(question)
print(f"Chain response: {response}")

Chain response: The CEO of Pineapple Technologies is Aurora Finch.


# The agent so far is more like a Q&A style.. If there is a follow up question -- Agent doesn't have the knowledge of previous questions and would give a wrong answer

we should build a conversational style agent

In [39]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response),
])

print(chat_history)


[HumanMessage(content='Who is the CEO of Pineapple Inc?', additional_kwargs={}, response_metadata={}), AIMessage(content='The CEO of Pineapple Technologies is Aurora Finch.', additional_kwargs={}, response_metadata={})]


In [42]:
from langchain_core.prompts import MessagesPlaceholder
context_query_system_prompt = (
    """ Given some chat history and a latest question which might refer to previous questions, 
    formulate a standalone question that can be answered using the provided context.
    Do not answer the question directly, just reformulate it if needed and otherwise return as-is.
    """
)

context_query_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", context_query_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualized_chain = context_query_prompt | llm | output_parser
contextualized_chain.invoke({"input": "Where is he located?", "chat_history": chat_history})


'Where is Aurora Finch, the CEO of Pineapple Technologies, located?'

In [46]:
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

history_aware_retriever = create_history_aware_retriever(llm, retriever, context_query_prompt)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided context."),
    ("system", "Context: {context}"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

rag_chain.invoke({
    "input": "Where is it located?",
    "chat_history": chat_history
})

{'input': 'Where is it located?',
 'chat_history': [HumanMessage(content='Who is the CEO of Pineapple Inc?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The CEO of Pineapple Technologies is Aurora Finch.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='5563a218-c4f8-4226-875c-0cc0989dc57d', metadata={'source': '../docs/pineapple_company.docx'}, page_content='Pineapple Technologies Overview'),
  Document(id='4f336d61-9ca1-4ae8-8d18-61977ffbc9ba', metadata={'source': '../docs/pineapple_company.docx'}, page_content='Pineapple Technologies is a fictional tech giant based in Redwood Valley, California.\nFounded in 1982 by Lana Swift, Igor Dusk, and Martina Vale, the company rose to fame with its Pi-Phone series.\nMilestones include the invention of the NeuralPalm interface (1995), the PineTab (2004), and the launch of the PiOS quantum chip (2022).\nCEO Aurora Finch took the helm in 2018, guiding the company through a major holographic transitio